## Week 11 Homework 2: Generating Answers: Input Text ==> Chunking ==> Embedding ==> Search Index ==> Query ==> Search ==> Question ==> Answer

In [52]:
question = "What are the main vector database providers in the Generative AI?"

In [53]:
text = """
Generative AI: Comprehensive Landscape Overview

Core Concepts
Foundational Technologies
Large Language Models (LLMs): AI systems trained on massive text datasets to generate human-like text.
Transformer Architecture: Neural network design enabling advanced language processing tasks.
Generative AI: AI systems capable of creating new content across various formats (text, images, audio, video).

Model Types
Text Generation Models
GPT (OpenAI): Popular model for natural language generation and completion.
Claude (Anthropic): Ethical AI-focused LLM for dialogue and reasoning.
LLaMA (Meta): Open-source LLM for research and commercial use.
Gemini (Google): Google's flagship language model for enterprise use.
Mistral: Lightweight and efficient models for diverse applications.
Image Generation Models
DALL-E (OpenAI): Generates images from textual descriptions.
Midjourney: Specialized in high-quality artistic and realistic image generation.
Stable Diffusion: Open-source model for customizable image generation.
Imagen (Google): High-resolution text-to-image generation model.
Multimodal Models
GPT-4: Combines text and image inputs for versatile AI applications.
Claude 3: Enhances multimodal understanding for complex tasks.
Gemini Pro: Designed for robust multimodal use cases in enterprise AI.

Key Libraries & Frameworks
Python-Based AI Development
LangChain: Framework for chaining LLM calls and developing AI applications.
Transformers (Hugging Face): Comprehensive library for working with pre-trained LLMs.
PyTorch: Flexible and widely-used deep learning framework.
TensorFlow: Google's machine learning platform for scalable model development.
Model Deployment
Ollama: Enables local model hosting and running.
Hugging Face: Centralized platform for hosting and sharing models.
MLflow: Streamlines the machine learning lifecycle, including tracking and deployment.
Inference Platforms
OpenAI API: Commercial access to advanced generative AI models.
Groq: High-speed inference platform for large models.
Anthropic API: Focused on ethical AI deployment.
Google Vertex AI: Comprehensive AI deployment for enterprises.

Specialized Tools
Development Tools
Weights & Biases: Tracks ML experiments and manages model versions.
Colab: Cloud-based interactive notebook for experimentation.
Kaggle: Platform for ML competitions and dataset sharing.
Fine-Tuning Tools
LoRA: Lightweight fine-tuning for LLMs using low-rank adaptation.
Keras: Simplifies neural network fine-tuning.
Hugging Face Accelerate: Optimizes distributed training for large models.

Data Storage and Retrieval Tools
Vector Databases & Semantic Search
Weaviate: Open-source vector database for storing and retrieving embeddings, supporting RAG workflows.
Features: Hybrid search, cloud-native design, multiple ML model support.
Pinecone: Managed vector database offering enterprise scalability and low-latency search.
Chroma: Open-source embedding database for ML applications and local storage.
Graph Databases
Neo4j: Advanced graph database for knowledge graphs, semantic search, and relationship modeling.
Use Case: Complex contextual understanding for generative AI.
Complementary Tools
Qdrant: High-performance vector similarity search engine.
Milvus: Scalable open-source vector database with multiple index types.

Architectural Integration
Pipeline Example:
Data Ingestion: Load documents into vector databases like Weaviate.
Embedding Creation: Convert text into vector representations.
Storage: Save embeddings in a vector database.
Query: Use semantic search for context retrieval.
Generation: LLM produces a response based on retrieved context.

Practical Application Domains
Natural Language Processing (NLP): Language understanding, summarization, translation.
Computer Vision: Image classification, generation, and object detection.
Code Generation: Auto-generating code for programming tasks.
Creative Content Production: Generating art, music, or stories.
Scientific Research: Data analysis and hypothesis testing.
Conversational AI: Chatbots and virtual assistants.
Personalized Recommendations: Customized suggestions for users.

Challenges in Generative AI
Computational Cost: Requires significant resources for training and inference.
Bias Mitigation: Addressing inherent biases in datasets.
Privacy Concerns: Protecting sensitive data in AI workflows.
Ethical Use: Ensuring responsible AI deployment.
Hallucination Prevention: Reducing instances of generating incorrect or nonsensical outputs.

Emerging Trends
Retrieval-Augmented Generation (RAG): Enhances LLMs with external knowledge.
Ethical AI Development: Focused on transparency and fairness.
Multimodal AI Integration: Combining text, images, and video for holistic AI solutions.
Edge AI: Running models locally for privacy and low latency.

Future Outlook
Increased Model Efficiency: Reducing computational requirements.
Specialized Domain Models: Tailored solutions for specific industries.
Enhanced Interpretability: Making AI decisions more transparent.
Stronger Ethical Frameworks: Ensuring responsible use of AI.
Democratization of AI Technologies: Making AI tools accessible to more users.

Key Platforms: LLaMA Index and Databricks
LLaMA Index (formerly GPT Index)
Purpose: Bridges custom data sources with LLMs.
Features:
Data ingestion from diverse formats (PDFs, APIs, web content).
Retrieval-Augmented Generation (RAG) for context-aware responses.
Use Cases: Custom AI chatbots, Q&A systems, and contextual applications.
Databricks
Purpose: Enterprise-grade platform for big data and AI.
Components:
Databricks Runtime: Spark-based large-scale data processing.
Delta Lake: Reliable data storage with ACID compliance.
MLflow: Manages the ML lifecycle.
Use Cases: Predictive analytics, machine learning, and enterprise AI deployments.

Comparative Ecosystem Strengths
Tool/Platform
Strengths
Weaviate
Flexibility, open-source
Pinecone
Scalability, enterprise-grade
Neo4j
Advanced relationship modeling
Chroma
ML workflow integration
Databricks
Big data and enterprise AI support
LLaMA Index
Simplified AI data connectivity

"""

# **SETUP**

In [54]:
!pip install cohere python-dotenv annoy
import os
import cohere
import warnings
import numpy as np
import pandas as pd
from annoy import AnnoyIndex
from dotenv import load_dotenv

# Suppress warnings
warnings.filterwarnings('ignore')

# Mount Google Drive (only necessary if working in Google Colab)
from google.colab import drive
drive.mount('/content/drive')

# Load environment variables from the specified .env file
load_dotenv('/content/drive/My Drive/key.env.txt')  # Update the path to your .env file

# Access the OpenAI API key
OPENAI_API = os.getenv('openai_api_key')
COHERE_API = os.getenv('cohere_api_key')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Chunking**

In [55]:
# Split into a list of paragraphs
texts = text.split('\n\n')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts if t])

In [56]:
texts[:3]

array(['Generative AI: Comprehensive Landscape Overview',
       'Core Concepts\nFoundational Technologies\nLarge Language Models (LLMs): AI systems trained on massive text datasets to generate human-like text.\nTransformer Architecture: Neural network design enabling advanced language processing tasks.\nGenerative AI: AI systems capable of creating new content across various formats (text, images, audio, video).',
       "Model Types\nText Generation Models\nGPT (OpenAI): Popular model for natural language generation and completion.\nClaude (Anthropic): Ethical AI-focused LLM for dialogue and reasoning.\nLLaMA (Meta): Open-source LLM for research and commercial use.\nGemini (Google): Google's flagship language model for enterprise use.\nMistral: Lightweight and efficient models for diverse applications.\nImage Generation Models\nDALL-E (OpenAI): Generates images from textual descriptions.\nMidjourney: Specialized in high-quality artistic and realistic image generation.\nStable Diffusi

# **Embedding**

In [57]:
co = cohere.Client(COHERE_API)
# Get the embeddings
response = co.embed(
    texts=texts.tolist(),
).embeddings

# **Build a search index**

In [58]:
# Check the dimensions of the embeddings
embeds = np.array(response)

# Create the search index, pass the size of embedding
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('test.ann')

True

# **Searching Article**

In [59]:
def search_document(query):
    # Get the query's embedding
    query_embed = co.embed(texts=[query]).embeddings

    # Retrieve the nearest neighbors
    similar_item_ids = search_index.get_nns_by_vector(query_embed[0],
                                                    10,
                                                  include_distances=True)

    search_results = texts[similar_item_ids[0]]

    return search_results

In [60]:
results = search_document(
    "Which companies provided system-wide orchestration for generative AI?"
)

print(results[0])

Key Libraries & Frameworks
Python-Based AI Development
LangChain: Framework for chaining LLM calls and developing AI applications.
Transformers (Hugging Face): Comprehensive library for working with pre-trained LLMs.
PyTorch: Flexible and widely-used deep learning framework.
TensorFlow: Google's machine learning platform for scalable model development.
Model Deployment
Ollama: Enables local model hosting and running.
Hugging Face: Centralized platform for hosting and sharing models.
MLflow: Streamlines the machine learning lifecycle, including tracking and deployment.
Inference Platforms
OpenAI API: Commercial access to advanced generative AI models.
Groq: High-speed inference platform for large models.
Anthropic API: Focused on ethical AI deployment.
Google Vertex AI: Comprehensive AI deployment for enterprises.


# **Generating Answers**

In [61]:
def ask_question_about_document(question, num_generations=1):

    # Search the text archive
    results = search_document(question)

    # Get the top result
    context = results[0]

    # Prepare the prompt
    prompt = f"""
    overview of generative AI landscape:
    {context}
    Question: {question}

    Extract the answer of the question from the text provided.
    If the text doesn't contain the answer,
    reply that the answer is not available."""

    prediction = co.generate(
        prompt=prompt,
        max_tokens=150,
        model="command-nightly",
        temperature=0.5,
        num_generations=num_generations
    )

    return prediction.generations

In [62]:
results = ask_question_about_document(
    "Which companies provided system-wide orchestration for generative AI?",
)

print(results[0].text)

The answer is not available. The text provided does not mention any companies offering system-wide orchestration for generative AI.


In [63]:
results = ask_question_about_document(
    "Are side projects a good idea when trying to build a career in AI?",
    num_generations=2
)

for gen in results:
    print(gen.text)
    print('--')

The text provided focuses on the practical application domains of Generative AI and does not specifically mention side projects. Therefore, the answer to your question is not directly available in the text.
--
The text provided focuses on the practical application domains of Generative AI and does not discuss the topic of side projects in relation to building a career in AI. Therefore, the answer is not available in the given text.
--


In [64]:
results = ask_question_about_document(
    "What are the fundamental tools to know in order to get started with generative AI?",
    num_generations=1
)

In [65]:
for gen in results:
    print(gen.text)

The text provides a comprehensive overview of the Generative AI landscape, but it does not specify the fundamental tools required to get started. Therefore, the answer is not directly available in the text provided.
